## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2025-10-10-20-50-33 +0000,nypost,"Gold could top $5K in a year, double that by 2...",https://nypost.com/2025/10/10/business/gold-co...
1,2025-10-10-20-48-00 +0000,wsj,"Trump Threatens Higher Tariffs on China, Citin...",https://www.wsj.com/politics/policy/trump-chin...
2,2025-10-10-20-47-32 +0000,startribune,Savannah Bananas’ baseball hijinks are coming ...,https://www.startribune.com/savannah-bananas-t...
3,2025-10-10-20-46-59 +0000,nyt,"Nasty News? Until Now, the Markets Said All Wa...",https://www.nytimes.com/2025/10/10/business/ma...
4,2025-10-10-20-46-56 +0000,nypost,Bari Weiss looks to calm nerves at CBS News as...,https://nypost.com/2025/10/10/media/bari-weiss...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2346/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
6,trump,56
227,gaza,32
76,peace,26
75,nobel,24
66,new,22


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
165,2025-10-10-14-32-26 +0000,nypost,"Fresh off Nobel Peace Prize win, opposition le...",https://nypost.com/2025/10/10/world-news/nobel...,196
16,2025-10-10-20-25-53 +0000,nyt,María Corina Machado Wins Nobel Peace Prize De...,https://www.nytimes.com/2025/10/10/world/ameri...,170
93,2025-10-10-17-17-55 +0000,nypost,Nobel Peace Prize winner Maria Corina Machado ...,https://nypost.com/2025/10/10/us-news/nobel-pe...,158
33,2025-10-10-19-38-56 +0000,nyt,"Democrats Praise Israel-Gaza Peace Deal, but N...",https://www.nytimes.com/2025/10/10/us/politics...,147
64,2025-10-10-18-27-32 +0000,bbc,White House blasts Nobel Committee for not awa...,https://www.bbc.com/news/articles/c7842qg15p6o...,142


## Select top headlines

In [5]:
with open('exclude_sources.txt') as f:
    excluded_sources = {
        line.strip()
        for line in f
        if line.strip() and not line.strip().startswith('#')
    }

top_count = 10
top_rows = []
working = word_scores.copy()
remaining = latest.copy()

while len(top_rows) < top_count and not remaining.empty:
    scored_remaining = remaining.assign(
        score=remaining['title'].apply(
            lambda t: sum(
                working.get(w.lower(), 0)
                for w in re.findall(r'[A-Za-z]+', t)
                if len(w) > 1
            )
        )
    )
    ranked_loop = scored_remaining.sort_values('score', ascending=False)

    if ranked_loop.empty:
        break

    top_story = ranked_loop.iloc[0]
    remaining = remaining.drop(top_story.name)

    if top_story['source'] in excluded_sources:
        continue

    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for word in words:
        working.pop(word, None)

    top_rows.append(top_story[['score', 'pubdate', 'source', 'title', 'link']])

top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
165,196,2025-10-10-14-32-26 +0000,nypost,"Fresh off Nobel Peace Prize win, opposition le...",https://nypost.com/2025/10/10/world-news/nobel...
333,96,2025-10-09-22-34-27 +0000,nypost,American military to oversee initial Gaza peac...,https://nypost.com/2025/10/09/us-news/american...
203,84,2025-10-10-11-33-02 +0000,nyt,"How Letitia James, Now Indicted, Will Stay as ...",https://www.nytimes.com/2025/10/10/us/politics...
98,55,2025-10-10-16-59-38 +0000,nypost,Trump threatens to scrap meeting with Chinese ...,https://nypost.com/2025/10/10/us-news/trump-th...
330,53,2025-10-09-22-42-00 +0000,wsj,Israeli Government Approves Hostage Deal Setti...,https://www.wsj.com/world/middle-east/israel-h...
312,46,2025-10-09-23-54-00 +0000,wsj,The Bureau of Labor Statistics will bring some...,https://www.wsj.com/politics/policy/federal-go...
161,44,2025-10-10-14-45-15 +0000,latimes,"Sister Jean, nun who became a national icon du...",https://www.latimes.com/sports/story/2025-10-1...
280,40,2025-10-10-03-22-00 +0000,wsj,"As tensions rise on Capitol Hill, Republican a...",https://www.wsj.com/politics/policy/government...
286,33,2025-10-10-02-31-37 +0000,nypost,"Cindy McCain, widow of Sen. John McCain and he...",https://nypost.com/2025/10/09/world-news/cindy...
336,32,2025-10-09-22-20-03 +0000,nypost,Turning Point USA to air its own ‘All American...,https://nypost.com/2025/10/09/us-news/turning-...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
